In [1]:
%matplotlib inline

In [2]:
import json
from tqdm import tqdm

from utils import *
import plotly_express as px
import plotly.graph_objects as go
import plotly as py
import matplotlib.pyplot as plt

In [3]:
sample = False

In [4]:
MAIN = pathlib.Path('/Users/palermopenano/personal/kaggle_energy')

In [5]:
d = pd.read_csv(MAIN / 'data' / 'train.csv')
d['timestamp'] = pd.to_datetime(d['timestamp'], infer_datetime_format=True)

In [6]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
building_id      int64
meter            int64
timestamp        datetime64[ns]
meter_reading    float64
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 616.9 MB


In [7]:
# Take only a random sample of n buildings
if sample:
    d, randbuilding = df_sample_random_buildings(d, 'building_id', n=10)
    d = d[d.timestamp < '2016-02-01']
    print(d.shape)

# Generate data for plot
* Transformations
    * Take average for the day
    * Linearly interpolate missing values between dates

In [8]:
def get_building_w_meter(meter_id):
    return [b for b in d.building_id.unique() if meter_id in d[d.building_id==b].meter.unique()]

In [9]:
# Aggregate by day (otherwise, Google Charts API fails)
dd = d.set_index('timestamp')
dd = dd.sort_index(ascending=True)
dd = dd.groupby(['building_id', 'meter'])['meter_reading'].resample('D').mean()
dd = dd.reset_index(['building_id', 'meter'])
d = dd

In [10]:
d.meter.unique()

array([0, 1, 3, 2])

In [11]:
# Get buildings with meter 0, 1, 2, or 3 only

meters = [0, 1, 2, 3]

for m in meters:
    print(f"Generating data for meter {m}...")
    m_data = []
    b_ids = get_building_w_meter(m)

    for b in tqdm(b_ids):
        b_data = {}
        b_data["b_id"] = int(b)
        cond = (d.building_id == b) & (d.meter == m)
        b_data["data"] = (
            d.loc[cond, 'meter_reading'].
            interpolate().  # linear interpolation to fill missing
            round(3).
            tolist()
        )
        m_data.append(b_data)

    dict_str = json.dumps(m_data, indent=4)
    js_data = f"var dataM{m} = " + dict_str
    p = pathlib.Path('/Users/palermopenano/Desktop/jh_ts_plot/Scripts')
    with open(p / f'dataM{m}.js', 'w') as fp:
        fp.write(js_data)
        
        

Generating data for meter 0...


100%|██████████| 1413/1413 [00:13<00:00, 105.50it/s]


Generating data for meter 1...


100%|██████████| 498/498 [00:04<00:00, 109.28it/s]


Generating data for meter 2...


100%|██████████| 324/324 [00:02<00:00, 109.71it/s]


Generating data for meter 3...


100%|██████████| 145/145 [00:01<00:00, 113.44it/s]


In [12]:
# d[d.building_id == 51]

# Plotly time series
* How to insert into HTML
https://stackoverflow.com/questions/36262748/python-save-plotly-plot-to-local-file-and-insert-into-html
* How to store underlying data separately and have javascript code using plotly reference that data ???
* How to stack figures in html code???

In [13]:
# buildings = d.building_id.unique()

# axis_settings = dict(
#     showgrid=False, 
#     zeroline=False, 
#     showline=False, 
#     ticks='', 
#     showticklabels=False
# )

# layout = go.Layout(
#     xaxis_title="",
#     yaxis_title="",
#     yaxis=axis_settings,
#     xaxis=axis_settings,
#     margin=dict(
#         l=0,
#         r=0,
#         b=0,
#         t=0,
#         pad=0
#     )
# )

# for b in tqdm(buildings[:2]):
#     meters = d[d.building_id == b].meter.unique()
#     print(b, meters)
#     for m in meters:
# #         print(b,m)
#         cond = (d.building_id == b) & (d.meter == m)
#         df = d[cond]
#         if len(df) == 0:
#             print(f"No data for {b} {m}")
#             continue
#         fig = go.Figure(
#             data=[go.Line(x=df['timestamp'], y=df['meter_reading'])],
#             layout=layout
#         )
# #         fig.show()
# #         py.offline.plot(fig, filename='fig.html')

#         # Generate
#         aa = py.offline.plot(fig, include_plotlyjs=False, output_type='div')
#         print(aa)
